In [1]:
import threading
import queue
import time
import random
from IPython.display import display

In [2]:
log_queue = queue.Queue()

def log_handler():
    """Thread that reads from the queue and prints messages safely."""
    while True:
        message = log_queue.get()
        if message is None:
            break
        display(message)
        log_queue.task_done()

def log(message):
    log_queue.put(message)

log_thread = threading.Thread(target=log_handler)
log_thread.start()

In [3]:
N = 5  # Number of philosophers

In [4]:
class Semaphore:
    def __init__(self, value=1):
        self.value = value
        self.condition = threading.Condition()

    def acquire(self):
        with self.condition:
            while self.value == 0:
                self.condition.wait()
            self.value -= 1

    def release(self):
        with self.condition:
            self.value += 1
            self.condition.notify_all()

In [5]:
class Mutex:
    def __init__(self):
        self.lock = threading.Condition()
        self.is_locked = False

    def acquire(self):
        with self.lock:
            while self.is_locked:
                self.lock.wait()
            self.is_locked = True

    def release(self):
        with self.lock:
            self.is_locked = False
            self.lock.notify_all()

In [6]:
def think(philosopher_id):
    log(f"Philosopher {philosopher_id} is thinking.")
    time.sleep(random.uniform(0.2, 0.5))

def eat(philosopher_id):
    log(f"Philosopher {philosopher_id} is eating.")
    time.sleep(random.uniform(0.2, 0.5))

In [7]:
# Solution 1: Using Semaphore
class PhilosophersSemaphoreStrategy:
    def __init__(self):
        self.forks = [Mutex() for _ in range(N)]
        self.room = Semaphore(N - 1)  # Only allow N-1 philosophers at a time

    def dine(self, philosopher_id, stop_event):
        first_fork, second_fork = philosopher_id, (philosopher_id + 1) % N
        while not stop_event.is_set():
            think(philosopher_id)

            self.room.acquire()
            self.forks[first_fork].acquire()
            self.forks[second_fork].acquire()

            eat(philosopher_id)

            self.forks[first_fork].release()
            self.forks[second_fork].release()
            self.room.release()

In [8]:
# Solution 2: Using Mutex
class PhilosophersMutexStrategy:
    def __init__(self):
        self.forks = [Mutex() for _ in range(N)]
        self.mutex = Mutex()

    def dine(self, philosopher_id, stop_event):
        first_fork, second_fork = philosopher_id, (philosopher_id + 1) % N
        while not stop_event.is_set():
            think(philosopher_id)

            self.mutex.acquire()
            if self.forks[first_fork].is_locked or self.forks[second_fork].is_locked:
                log(f"Philosopher {philosopher_id} failed to eat.")
                self.mutex.release()
                continue
            self.forks[first_fork].acquire()
            self.forks[second_fork].acquire()
            self.mutex.release()

            eat(philosopher_id)

            self.forks[first_fork].release()
            self.forks[second_fork].release()

In [9]:
class Simulation(threading.Thread):
    def __init__(self, strategy):
        super().__init__()
        self._stop_event = threading.Event()
        self.philosophers = [
            threading.Thread(target=strategy.dine, args=(i, self._stop_event))
            for i in range(N)
        ]

    def run(self):
        for p in self.philosophers:
            p.start()

    def stop(self):
        self._stop_event.set()
        for p in self.philosophers:
            p.join()

In [10]:
stg = PhilosophersSemaphoreStrategy()
simulation_thread = Simulation(stg)
simulation_thread.start()
time.sleep(2)

simulation_thread.stop()
simulation_thread.join()

'Philosopher 0 is thinking.'

'Philosopher 1 is thinking.'

'Philosopher 2 is thinking.'

'Philosopher 3 is thinking.'

'Philosopher 4 is thinking.'

...

'Philosopher 4 is eating.'

'Philosopher 2 is eating.'

'Philosopher 1 is eating.'

'Philosopher 3 is eating.'

'Philosopher 0 is eating.'

In [11]:
stg = PhilosophersMutexStrategy()
simulation_thread = Simulation(stg)
simulation_thread.start()
time.sleep(2)

simulation_thread.stop()
simulation_thread.join()

'Philosopher 0 is thinking.'

'Philosopher 1 is thinking.'

'Philosopher 2 is thinking.'

'Philosopher 3 is thinking.'

'Philosopher 4 is thinking.'

...

'Philosopher 4 is eating.'

'Philosopher 0 failed to eat.'

'Philosopher 1 is eating.'

'Philosopher 3 is eating.'

'Philosopher 2 failed to eat.'

In [12]:
# Stop the logger
log(None)
log_thread.join()